In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import helper_functions as hf
import pyarrow.parquet as pq

In [ ]:
yellow_data = hf.load_data('taxi_data/yellow_tripdata_2024-')

In [ ]:
green_data = hf.read_taxi_files('data_files/green_taxi_data/green_tripdata_2024-', 1)

In [9]:
green_data.columns

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge'],
      dtype='object')

In [3]:
fhv_data = hf.read_taxi_files('data_files/fhv_data/fhv_tripdata_2024-', 1)

In [4]:
fhv_data.columns

Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number'],
      dtype='object')

In [ ]:
yellow_data

In [ ]:
green_data

In [ ]:
fhv_data

In [ ]:
fhv_data['PUlocationID'].hist(bins=264)

In [ ]:
# fhv_data['PUlocationID'].value_counts()
PUlocationID_nan_count = fhv_data['PUlocationID'].isna().sum()

print(PUlocationID_nan_count)

In [ ]:
yellow_data['PULocationID'].hist(bins=264)

In [ ]:
green_data['PULocationID'].hist(bins=264)

In [ ]:
green_data.columns

In [ ]:
green_columns_to_drop = ['VendorID',
       'store_and_fwd_flag', 'RatecodeID', 
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge']

In [ ]:
yellow_data.columns

In [ ]:
columns_to_drop = ['VendorID', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee']

In [ ]:
relevant_data = yellow_data.drop(columns_to_drop, axis=1)

In [ ]:
relevant_data_green = green_data.drop(green_columns_to_drop, axis= 1)

In [ ]:
relevant_data_green

In [ ]:
relevant_data

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

# === 1. Wczytanie danych przejazdów ===
print("Wczytywanie danych przejazdów...")
df = relevant_data
# Wyciągnięcie godziny i dnia tygodnia
df["hour"] = df["tpep_pickup_datetime"].dt.hour
df["day_of_week"] = df["tpep_pickup_datetime"].dt.dayofweek

# Zliczanie przejazdów dla każdej strefy
pickup_counts = df.groupby("PULocationID").size().reset_index(name="ride_count")

# === 2. Wczytanie mapy NYC Taxi Zones ===
print("Wczytywanie mapy NYC Taxi Zones...")
zones = gpd.read_file("taxi_zones.shp")  # Pobierz plik z https://www.nyc.gov/assets/tlc/downloads/pdf/taxi_zone_shapefile.zip

# Sprawdzenie zgodności kluczy
zones["LocationID"] = zones["LocationID"].astype(int)

# Połączenie liczby przejazdów z mapą
zones = zones.merge(pickup_counts, left_on="LocationID", right_on="PULocationID", how="left")
zones["ride_count"] = zones["ride_count"].fillna(0)  # Brakujące wartości → 0 kursów

# === 3. Tworzenie heatmapy ===
print("Tworzenie heatmapy...")
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

# Rysowanie mapy z kolorem oznaczającym liczbę kursów
zones.plot(column="ride_count", cmap="Reds", linewidth=0.5, edgecolor="black", legend=True, ax=ax)

# Dodanie tytułu
plt.title("Heatmapa liczby przejazdów NYC Taxi - Manhattan", fontsize=14)
plt.axis("off")  # Ukrycie osi

# Wyświetlenie mapy
plt.show()


In [ ]:
df["hour"] = df["tpep_pickup_datetime"].dt.hour
df["day_of_week"] = df["tpep_pickup_datetime"].dt.dayofweek

In [ ]:
df.head(10)

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

for i in range(2,24):
    
    filtered = df.query('hour == ' +str(i))
    # Zliczanie przejazdów dla każdej strefy
    pickup_counts = filtered.groupby("PULocationID").size().reset_index(name="ride_count")

    # === 2. Wczytanie mapy NYC Taxi Zones ===
    print("Wczytywanie mapy NYC Taxi Zones...")
    zones = gpd.read_file("taxi_zones.shp")

    # Sprawdzenie zgodności kluczy
    zones["LocationID"] = zones["LocationID"].astype(int)

    # Połączenie liczby przejazdów z mapą
    zones = zones.merge(pickup_counts, left_on="LocationID", right_on="PULocationID", how="left")
    zones["ride_count"] = zones["ride_count"].fillna(0)  # Brakujące wartości → 0 kursów

    # === 3. Tworzenie heatmapy ===
    print("Tworzenie heatmapy...")
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))

    # Rysowanie mapy z kolorem oznaczającym liczbę kursów
    zones.plot(column="ride_count", cmap="Reds", linewidth=0.5, edgecolor="black", legend=True, ax=ax)

    # Dodanie tytułu
    plt.title(f"Heatmapa liczby przejazdów NYC Taxi - Manhattan h = {i}", fontsize=14)
    plt.axis("off")  # Ukrycie osi

    # Wyświetlenie mapy
    # plt.show()


    plt.savefig('map_hour_'+ str(i) + '.png')


In [ ]:
import geopandas as gpd
import pandas as pd


# === 1. Wczytanie danych przejazdów ===
print("Wczytywanie danych przejazdów...")
df = relevant_data_green
# Wyciągnięcie godziny i dnia tygodnia
df["hour"] = df["lpep_pickup_datetime"].dt.hour
df["day_of_week"] = df["lpep_pickup_datetime"].dt.dayofweek

# Zliczanie przejazdów dla każdej strefy
pickup_counts = df.groupby("PULocationID").size().reset_index(name="ride_count")

# === 2. Wczytanie mapy NYC Taxi Zones ===
print("Wczytywanie mapy NYC Taxi Zones...")
zones = gpd.read_file("taxi_zones.shp")  # Pobierz plik z https://www.nyc.gov/assets/tlc/downloads/pdf/taxi_zone_shapefile.zip

# Sprawdzenie zgodności kluczy
zones["LocationID"] = zones["LocationID"].astype(int)

# Połączenie liczby przejazdów z mapą
zones = zones.merge(pickup_counts, left_on="LocationID", right_on="PULocationID", how="left")
zones["ride_count"] = zones["ride_count"].fillna(0)  # Brakujące wartości → 0 kursów

# === 3. Tworzenie heatmapy ===
print("Tworzenie heatmapy...")
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

# Rysowanie mapy z kolorem oznaczającym liczbę kursów
zones.plot(column="ride_count", cmap="Reds", linewidth=0.5, edgecolor="black", legend=True, ax=ax)

# Dodanie tytułu
plt.title("Heatmapa liczby przejazdów NYC Taxi - Manhattan", fontsize=14)
plt.axis("off")  # Ukrycie osi

# Wyświetlenie mapy
plt.show()



In [ ]:
relevant_data_green['PULocationID'].value_counts()